In [ ]:
# Libraries
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import random
import numpy as np

# Standard approach (de-identified experts)

In [ ]:
class Linear_net_rej(nn.Module):
    '''
   (Mozannar & Sontag) Linear classifier and deferral for L_CE loss for binary response
   novel convex consistent surrogate loss
    '''
    def __init__(self, input_dim, out_dim):
        super(Linear_net_rej, self).__init__()
        # an affine operation: y = Wx + b
        self.fc = nn.Linear(input_dim, out_dim+1) # out: 0,1,2
        self.fc_rej = nn.Linear(input_dim, 1)
        torch.nn.init.ones_(self.fc.weight)
        torch.nn.init.ones_(self.fc_rej.weight)
        self.softmax = nn.Softmax(dim=0) # dim = 0 to get 0,1,2 as output

    def forward(self, x):
        out = self.fc(x)
        rej = self.fc_rej(x)
        #out = torch.cat([out,rej],1)
        out = self.softmax(out)
        return out

In [ ]:
def reject_CrossEntropyLoss(outputs, h, labels, m, n_classes):
    '''
    (Mozannar & Sontag) Implmentation of L_{CE}^{\alpha}
        outputs: classifier and rejector model outputs
        h: cost of deferring to expert cost of classifier predicting (I_{m =y})
        labels: target
        m: cost of classifier predicting (alpha* I_{m\neq y} + I_{m =y})
        n_classes: number of classes
    '''    
    batch_size = outputs.size()[0]            # batch_size
    rc = torch.tensor([n_classes] * batch_size, dtype=torch.long)
    #labels = torch.tensor(labels, dtype=torch.long)
    labels = labels.clone().detach().long()
    outputs =  -h*torch.log2( outputs[range(batch_size), rc]) - m*torch.log2(outputs[range(batch_size), labels])   # pick the values corresponding to the labels
    return torch.sum(outputs)/batch_size

In [ ]:
def run_classifier_rej(model, exp_heuristic, data_x, data_y, alpha, p, k):
    '''
    (Mozannar & Sontag) training script for L_{CE}
        model: classifier and rejector model
        data_x: input
        data_y: label
        alpha: hyperparam alpha for loss L_CE^{\alpha}
        p: probability of randomly selecting expert 1
    '''
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.5, 0.99), weight_decay=1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(data_x)*100)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)

    for epoch in range(1):  # loop over the dataset multiple times
        inputs = torch.tensor(data_x)
        labels = torch.tensor(data_y)

        # split to create batch size
        x_batches = torch.split(inputs, 5)
        y_batches = torch.split(labels, 5) 

        loss_train = []

        model_predictions = 0
        expert_predictions = 0
        total_samples = 0


        for inputs, labels in zip(x_batches, y_batches):

            optimizer.zero_grad()

            # forward + backward + optimize
            rand_exp = random.choices([1, k], weights=[p, 1-p])[0] # randomly select expert k

            predicted = torch.tensor(exp_heuristic[rand_exp-1](inputs))  # get predictions from selected expert
            
            h = (predicted==labels)*1
            m = [0] * len(inputs) 
            for j in range (0,len(inputs)): # determines weights
                if h[j]:
                    m[j] = alpha
                    expert_predictions += 1
                else:
                    m[j] = 1

            h = h.clone().detach()
            m = torch.tensor(m)
            inputs = inputs.to(model.fc.weight.dtype)
            outputs = model(inputs)

            # Loss
            loss = reject_CrossEntropyLoss(outputs, h, labels, m, 2) # this is loss for classifier and rejector
            loss.backward()
            optimizer.step()
            scheduler.step()
            loss_train.append(loss.item())


            # Training accuracy
            model_predictions += torch.sum((torch.argmax(outputs, dim=1) == labels).float()).item()
            expert_predictions += torch.sum((torch.argmax(predicted) == labels).float()).item()
            total_samples += len(labels)
            #model_accuracy = model_predictions / total_samples
            #expert_accuracy = expert_predictions / total_samples

    #print('Model Training Accuracy: ', model_accuracy)
    #print('Overall Expert Training Accuracy:', expert_accuracy)

    plt.plot(loss_train, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()
        
            #print("loss " + str(loss.item()))

In [ ]:
def test_classifier_rej(model, exp_heuristic, data_x, data_y, p, k, exp_instances):
    '''
    (Mozannar & Sontag) Test classifier and deferral model for L_{CE} loss
    '''
    correct = 0
    correct_sys = 0
    exp = [0]*k
    exp_total = [0]*k
    total = 0
    real_total = 0
    points = len(data_x)

    with torch.no_grad():
        inputs = torch.tensor(data_x)
        labels = torch.tensor(data_y)

        inputs = inputs.to(model.fc.weight.dtype)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1) #0/1 (ML), 2 (defer to expert)
        predicted_exp = [torch.tensor(exp_heuristic(inputs)) for exp_heuristic in exp_heuristic]

        for i in range(len(inputs)):
            r = (predicted[i] == 2).item() # if 2, then defer to expert
            if r:
                rand_exp = random.choices([1, k], weights=[p, 1-p])[0] # randomly select expert for each point
                correct_sys += (predicted_exp[rand_exp-1][i] == labels[i]).item()
                exp[rand_exp-1] += (predicted_exp[rand_exp-1][i] == labels[i]).item()
                exp_total[rand_exp-1] += 1
                exp_instances[rand_exp-1].append(i)
            else: 
                correct += (predicted[i] == labels[i]).item()
                correct_sys += (predicted[i] == labels[i]).item()
                total += 1
        real_total += labels.size(0)

    print("system accuracy", 100 * correct_sys / real_total)
    print("total points:", points)
    print()

    for idx, (c, expert_total) in enumerate(zip(exp, exp_total)):
        print(f"Expert {idx+1} defer count:", expert_total)
        print(f"Expert {idx+1} defer percent:", 100 * expert_total / points if expert_total != 0 else 0)
        print(f"Expert {idx+1} correct predictions:", c)
        print(f"Expert {idx+1} accuracy:", 100 * c / (expert_total + 0.0002) if expert_total != 0 else 0)
        print()

    print("Not deferred to any expert count:", total)
    print("Not deferred percent:", 100 * total / points)
    print("Model correct predictions:", correct)
    print("Model accuracy:", 100 * correct / (total + 0.0001))
    print()

    #ratios = [exp_total / exp_correct if exp_correct != 0 else 0 for exp_total, exp_correct in zip(exp_total, exp)]
    #for idx, ratio in enumerate(ratios):
        #print(f"Expert {idx+1} to other experts ratio:", ratio)

    print()
    overall_exp_total = sum(exp_total)
    overall_exp_correct = sum(exp)
    print("Overall expert count:", overall_exp_total)
    print("Overall expert defer percent:", 100 * overall_exp_total / points)
    print("Overall expert correct predictions:", overall_exp_correct)
    print("Overall expert accuracy:", 100 * overall_exp_correct / (overall_exp_total + 0.0001) if overall_exp_total != 0 else 0)

    return exp_instances

# Optimal approach (identified experts)

In [ ]:
class Linear_net_rej_id(nn.Module):
    '''
   (Mozannar & Sontag) Linear classifier and deferral for L_CE loss for binary response
   novel convex consistent surrogate loss
    '''
    def __init__(self, input_dim, out_dim, k):
        super(Linear_net_rej_id, self).__init__()
        # an affine operation: y = Wx + b
        self.fc = nn.Linear(input_dim, out_dim+k) # out: 0,1,2,3
        self.fc_rej = nn.Linear(input_dim, 1)
        torch.nn.init.ones_(self.fc.weight)
        torch.nn.init.ones_(self.fc_rej.weight)
        self.softmax = nn.Softmax(dim=0) # dim = 0 to get 0,1,2,3 as output

    def forward(self, x):
        out = self.fc(x)
        rej = self.fc_rej(x)
        #out = torch.cat([out,rej],1)
        out = self.softmax(out)
        return out

In [ ]:
def reject_CrossEntropyLoss_id(outputs, h, labels, m, n_classes):
    '''
    (Mozannar & Sontag) Implmentation of L_{CE}^{\alpha}
        outputs: classifier and rejector model outputs
        h: cost of deferring to expert k cost of classifier predicting (I_{m =y})
        labels: target
        m: cost of classifier predicting (alpha* I_{m\neq y} + I_{m =y})
        n_classes: number of classes, binary here
    '''    
    batch_size = outputs.size()[0]            # batch_size
    rc = torch.tensor([n_classes] * batch_size, dtype=torch.long)
    labels = labels.clone().detach().long()
    outputs_exp = [torch.zeros(batch_size) for _ in range(k)]

    for i in range(k):
        outputs_exp[i] = -h[i] * torch.log2(outputs[range(batch_size), rc]) - m[:, i] * torch.log2(outputs[range(batch_size), labels])

    outputs = sum(outputs_exp)  # Sum the losses from all experts
    
    return torch.sum(outputs) / batch_size

In [ ]:
def run_classifier_rej_id(model, exp_heuristic, data_x, data_y, alpha, k):
    '''
    (Mozannar & Sontag) training script for L_{CE}
        model: classifier and rejector model
        data_x: input
        data_y: label
        alpha: expert 1 hyperparam alpha for loss L_CE^{\alpha} 
        beta: expert 2 hyperparam alpha for loss L_CE^{\beta} 
        p: probability of randomly selecting expert 1
    '''
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.08, betas = (0.75, .1), weight_decay=.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(data_x)*100)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3.0)

    for epoch in range(1):  # loop over the dataset multiple times
        inputs = torch.tensor(data_x)
        labels = torch.tensor(data_y)

        # split to create batch size
        x_batches = torch.split(inputs, 5)
        y_batches = torch.split(labels, 5) 

        loss_train = []

        for inputs, labels in zip(x_batches, y_batches):

            optimizer.zero_grad()

            predicted = [torch.tensor(expert(inputs)) for expert in exp_heuristic]

            h_list = [(expert_output == labels) * 1 for expert_output in predicted]

            m = torch.zeros(len(inputs), k)

            for j in range(len(inputs)):
                for i in range(k):
                    if h_list[i][j]:
                        m[j][i] = alpha
                    else:
                        m[j][i] = 1

            h_list = [h.clone().detach() for h in h_list]
            m = m.clone().detach()

            inputs = inputs.to(model.fc.weight.dtype)
            outputs = model(inputs)

            # Loss computation
            loss = reject_CrossEntropyLoss_id(outputs, h_list, labels, m, 2)
            loss.backward()
            optimizer.step()
            scheduler.step()
            loss_train.append(loss.item())

    plt.plot(loss_train, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.show()
        

In [ ]:
def test_classifier_rej_id(model, exp_heuristic, data_x, data_y, k, exp_instances):
    '''
    (Mozannar & Sontag) Test classifier and deferral model for L_{CE} loss
    '''
    correct = 0
    correct_sys = 0
    exp = [0]*k
    exp_total = [0]*k
    total = 0
    real_total = 0
    points = len(data_x)

    with torch.no_grad():
        inputs = torch.tensor(data_x)
        labels = torch.tensor(data_y)

        inputs = inputs.to(model.fc.weight.dtype)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predicted_exp = [torch.tensor(exp_heuristic(inputs)) for exp_heuristic in exp_heuristic]

        for i in range(len(inputs)):
            r = predicted[i].item() > 1  # if greater than 1, then defer to experts
            if r:
                expert_id = predicted[i].item() - 2  # expert index starts from 0
                correct_sys += (predicted_exp[expert_id][i] == labels[i]).item()
                exp[expert_id] += (predicted_exp[expert_id][i] == labels[i]).item()
                exp_total[expert_id] += 1
                exp_instances[expert_id].append(i)

            else:
                correct += (predicted[i] == labels[i]).item()
                correct_sys += (predicted[i] == labels[i]).item()
                total += 1
        real_total += labels.size(0)

    print("system accuracy", 100 * correct_sys / real_total)
    print("total points:", points)
    print()

    for idx, (c, expert_total) in enumerate(zip(exp, exp_total)):
        print(f"Expert {idx+1} defer count:", expert_total)
        print(f"Expert {idx+1} defer percent:", 100 * expert_total / points if expert_total != 0 else 0)
        print(f"Expert {idx+1} correct predictions:", c)
        print(f"Expert {idx+1} accuracy:", 100 * c / (expert_total + 0.0002) if expert_total != 0 else 0)
        print()

    print("Not deferred to any expert count:", total)
    print("Not deferred percent:", 100 * total / points)
    print("Model correct predictions:", correct)
    print("Model accuracy:", 100 * correct / (total + 0.0001))
    print()

    #ratios = [exp_total / exp_correct if exp_correct != 0 else 0 for exp_total, exp_correct in zip(exp_total, exp)]
    #for idx, ratio in enumerate(ratios):
        #print(f"Expert {idx+1} to other experts ratio:", ratio)

    print()
    overall_exp_total = sum(exp_total)
    overall_exp_correct = sum(exp)
    print("Overall expert count:", overall_exp_total)
    print("Overall expert defer percent:", 100 * overall_exp_total / points)
    print("Overall expert correct predictions:", overall_exp_correct)
    print("Overall expert accuracy:", 100 * overall_exp_correct / (overall_exp_total + 0.0001) if overall_exp_total != 0 else 0)

    return exp_instances


# Experimental setup

- different types of experts
    1. manually create heuristics
    2. experts as non-linear classifiers (decision tree classifiers)
    3. based on covariance instead of labels 
<br>
<br>
- different types of datasets
    1. complex, non-linearly separable (3 blobs)
<br>
<br>
- semi-synthetic setting
    1. hate speech detection: https://github.com/t-davidson/hate-speech-and-offensive-language
    2. fact-checking (Christo)
    3. CheXpert: https://stanfordmlgroup.github.io/competitions/chexpert/
    4. multi-label datasets for semantic scene and text classification: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multilabel.html
<br>
<br>
- alternative method
    1. majority vote? 
    2. hierarchical (highest rank/authority)
    3. panel of very similar views/knowledges
    4. more representative group (different views/knowledge sand various ranks/authority)



In [ ]:
'''
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, stratify=y, random_state=456)
'''

In [ ]:
'''
Standard approach

experts = [expert1, expert2]
k = len(experts)
m = Linear_net_rej(2,k) # 2 inputs
alpha = 0
p = 0.5

run_classifier_rej(m, experts, X_train, y_train, alpha, p, k)

# keep track of what in/correct instances are deferred to each expert by index and inputs
exp_index = [[] for _ in range(k)]

exp1_index, exp2_index = test_classifier_rej(m, experts, X_test, y_test, p, k, exp_index)

'''

In [ ]:
'''
Optimal approach

m = Linear_net_rej_id(2, 2, k) # 2 inputs
alpha = 0

run_classifier_rej_id(m, experts, X_train, y_train, alpha, k)

# keep track of what in/correct instances are deferred to each expert by index and inputs
exp_index = [[] for _ in range(k)]

exp1_index, exp2_index = test_classifier_rej_id(m, experts, X_test, y_test, k, exp_index)
'''